# Sistema Multiagentes para Criação de Roteiros de Viagem 🏖️

Criaremos um conjunto de agentes (crew) para definir um roteiro de viagem para um lugar escolhido, que tenha um bom custo-benefício de estadia e alimentação sendo também romantico para casais.

### **Sobre o CrewAI**:
> CrewAI é um framework de orquestração de IA de código aberto que permite a criação de *"tripulações" (crews)* de agentes autônomos que colaboram para completar tarefas complexas.

> Ele usa uma estrutura modular onde cada agente tem uma missão, ferramentas e uma função específica, e as tarefas podem ser delegadas entre eles. O objetivo é automatizar fluxos de trabalho e otimizar a resolução de problemas, simulando uma equipe de trabalho humana com agentes de IA.



# 1. Instalação do CREWAI


* O ```!``` é só pra dizer que é um comando de terminal.

* ```pip install crewai``` baixa e instala a lib.

* O ```--quiet``` deixa tudo mais silencioso, sem aquele monte de texto na tela.




In [1]:
!pip install --quiet crewai

# 2. Importação das dependências e variáveis de ambiente


* Estamos importando as peças principais do crewai.

  `from crewai import Agent, Task, Crew`

  * Agent → o “agente”, quem faz as tarefas.
  * Task → a tarefa que ele precisa executar.
  * Crew → o grupo que junta tudo e coordena a brincadeira.


* Importando o módulo do Colab para pegar as variáveis de ambiente.
  
  `from google.colab import userdata`

* Pegamos a chave da API do Gemini que foi salva no Colab.
  
  `GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')`


In [2]:
from crewai import Agent, Task, Crew
from google.colab import userdata

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

## Importamos também o GEMINI, modelo LLM da Google.

> Por padrão o CREWAI utiliza o GPT da OpenAI. Mas o Gemini tem mais créditos gratuitos pra nós meras mortais.

In [3]:
from crewai import LLM

gemini_llm = LLM(
    model="gemini/gemini-2.5-flash",
    api_key=GEMINI_API_KEY,
    temperature=0.0
)

# 2. Definição do Agente

Armazenamos em uma variável a entidade `Agent` para criar nosso primeiro agente, suas características e atribuições.

Aqui vamos usar os seguintes parâmetros:

1. **`role`**

É o *papel* do agente, o “título do cargo”.

2. **`goal`**

É o **objetivo principal** do agente — aquilo que ele sempre tenta alcançar.
Utilizamos variáveis (`{destino}` e `{tempo}`) para preencher na hora de executar o agente.

3. **`backstory`**

  É o **contexto**, a história por trás do agente. "Passado do vilão".
Vai guiar o modelo a ter um tom específico ou considerar responder como alguem com esse contexto responderia.

4. **`llm`**

  Aqui dizemos pra ele qual LLM usar, por default é o GPT mas vamos usar o GEMINI.

In [4]:
guia_turistico = Agent(
    role="Guia Turistico",
    goal="Entregar um roteiro de viagem para {destino} com o melhor custo benefício dado tempo {tempo} de férias.",
    backstory="Você é um guia turistico com 10 anos de experiência especialista em viagens para casais.",
    llm=gemini_llm
)

# 3. Definição da Tarefa

Agora armazenamos em uma variável a entidade `Task` para criar estrutura e regras que nosso agente vai executar na hora de tocar essa Tarefa.

Os parâmetros que usamos são:

1. **`agent`**

Quem vai executar essa tarefa.  

2. **`description`**

É onde você explica passo a passo o que você quer que o agente faça.

3. **`expected_output`**

Aqui você define como quer que a resposta final venha formatada.

In [5]:
escrever_roteiro = Task(
    agent=guia_turistico,
    description="""
    Escreva um roteiro de viagem para {destino}.
    Considere as melhores opções de restaurante considerando o melhor custo benefício.
    Inclua também opções de experiências nativas do local {destino}.
    E encaixe tudo no tempo disponível que é {tempo}.
    """,
    expected_output="Lista de restaurantes e experiências com horários definidos para cada um."
)

# 4. Definição do CREW


1. **`agents`**

Aqui você lista[] quais agentes fazem parte da equipe. Montamos o time.

2. **`tasks`**

Uma lista[] de quais tarefas esse time precisa executar.

3. **`verbose=True`**

Esse parâmetro serve pra mostrar pra gente o que está acontecendo por trás dos panos.

In [6]:
crew = Crew(
    agents=[guia_turistico],
    tasks=[escrever_roteiro],
    verbose=True
)

# 5. Fazendo funcionar pela primeira vez

Vamos rodar nosso crew pela primeira vez.
Esperamos que ao fim do pipeline, ele seja capaz de nos entregar uma lista com um belo roteiro de viagem seguindo o que pedimos para ele executar como Task.

Você lembra que na definição da tarefa e do agente colocamos entre chaves `{ }` os parâmetros que passariamos porteriormente para o Agente? Essa é a hora.

In [7]:
payload = {
    "destino": "Alagoas",
    "tempo": "01 semana"
}

roteiro_pronto = crew.kickoff(inputs=payload)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 33d9f482-ac2d-4cec-9bff-5746ca28854d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Guia Turistico                                                                                          │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Escreva um roteiro de viagem para Alagoas.                                                                 │
│      Considere as melhores opções de restaurante considerando o melhor custo benefício.                         │
│      Inclua também opções de experiências nativas do local Alagoas.                                             │
│      E encaixe tudo no tempo disponível que é 01 semana.                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Guia Turistico                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Olá, casal! Que alegria saber que escolheram Alagoas para uma semana de férias. Com meus 10 anos de            │
│  experiência em roteiros para casais, preparei algo especial para vocês, unindo o melhor do paraíso alagoano    │
│  com experiências autênticas e, claro, um excelente custo-benefício. Preparem-se para se apaixonar!             │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## Roteiro de Viagem para Casais em Alagoas: 01 Semana de Encantos e Sabores                                   │
│                                                                                                                 │
│  **Base:** Maceió (sugestão de hospedagem nas regiões de Ponta Verde, Pajuçara ou Jatiúca para fácil acesso às  │
│  praias e restaurantes).                                                                                        │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### **Dia 1: Chegada em Maceió e Primeiros Encantos da Orla**                                                  │
│                                                                                                                 │
│  *   **Manhã (09:00 - 12:00):** Chegada ao Aeroporto Zumbi dos Palmares (MCZ). Traslado para o hotel em         │
│  Maceió. Façam o check-in, deixem as malas e respirem o ar do paraíso!                                          │
│  *   **Almoço (13:00 - 14:30):** **Restaurante: Bodega do Sertão** (Av. Dr. Júlio Marques Luz, 220 - Jatiúca).  │
│      *   **Custo-benefício:** Médio-Alto. É um buffet por quilo, mas a variedade e a qualidade da comida        │
│  nordestina são espetaculares. É uma experiência cultural e gastronômica que vale cada centavo para o primeiro  │
│  contato com a culinária local.                                                                                 │
│  *   **Tarde (15:00 - 18:00):** **Experiência Nativa: Caminhada Romântica pela Orla de Ponta Verde e            │
│  Pajuçara.**                                                                                                    │
│      *   Aproveitem para sentir a brisa, tirar fotos com os coqueiros e observar as jangadas. É o cartão        │
│  postal de Maceió!                                                                                              │
│  *   **Fim de Tarde (18:00 - 19:00):** **Experiência Nativa: Visita à Feira de Artesanato da Pajuçara.**        │
│      *   Ótimo para encontrar lembrancinhas autênticas, como a renda filé, e conhecer o trabalho dos artesãos   │
│  locais.                                                                                                        │
│  *   **Jantar (20:00 - 21:30):** **Restaurante: Lopana** (Av. Silvio Carlos Viana, 27 - Ponta Verde).           │
│      *   **Custo-benefício:** Médio. Ambiente super agr

# 6. Vendo o resultado do Agente de Viagens.

No print abaixo pudemos ver que o Agente ficou inspirado com a nossa viagem e nos deu todas as dicas para um roteiro relevante de viagens.

Isso apenas baseado no que ele tem disponível dentro da LLM para trabalhar.

In [8]:
print(roteiro_pronto)

Olá, casal! Que alegria saber que escolheram Alagoas para uma semana de férias. Com meus 10 anos de experiência em roteiros para casais, preparei algo especial para vocês, unindo o melhor do paraíso alagoano com experiências autênticas e, claro, um excelente custo-benefício. Preparem-se para se apaixonar!

---

## Roteiro de Viagem para Casais em Alagoas: 01 Semana de Encantos e Sabores

**Base:** Maceió (sugestão de hospedagem nas regiões de Ponta Verde, Pajuçara ou Jatiúca para fácil acesso às praias e restaurantes).

---

### **Dia 1: Chegada em Maceió e Primeiros Encantos da Orla**

*   **Manhã (09:00 - 12:00):** Chegada ao Aeroporto Zumbi dos Palmares (MCZ). Traslado para o hotel em Maceió. Façam o check-in, deixem as malas e respirem o ar do paraíso!
*   **Almoço (13:00 - 14:30):** **Restaurante: Bodega do Sertão** (Av. Dr. Júlio Marques Luz, 220 - Jatiúca).
    *   **Custo-benefício:** Médio-Alto. É um buffet por quilo, mas a variedade e a qualidade da comida nordestina são espe

Mas, e se ele tivesse acesso a internet para sugerir coisas ainda mais relevantes que pedirmos?

# 7. Turbinando nosso Crew com o uso de TOOLS (ferramentas)

Vamos usar o SerperDev para fazer com que o nosso Agente pesquise na internet ao invés de contar com a resposta treinada do conhecimento da LLM.

In [9]:
!pip install --quiet crewai[tools]

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✅ Crew: crew
└── 📋 Task: 037fdd53-8086-4b75-b9bd-bfd5a2d1de01
    Assigned to: Guia Turistico
    Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 037fdd53-8086-4b75-b9bd-bfd5a2d1de01                                                                     │
│  Agent: Guia Turistico                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [10]:
from crewai_tools import SerperDevTool
import os

search_tool = SerperDevTool()
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [11]:
guia_turistico_turbinado = Agent(
    role="Guia Turistico",
    goal="""
    Entregar um roteiro de viagem para {destino} com as melhores indicações e lugares atualizados.
    Pesquise na internet pousadas com melhor custo benefício em {destino}
    dado tempo {tempo} de férias.
    """,
    backstory="Você é um guia turistico com 10 anos de experiência especialista em viagens para casais.",
    llm=gemini_llm,
    tools=[search_tool]
)

In [12]:
guia_gastronomico_turbinado = Agent(
    role="Guia Gastronomico",
    goal="""
    Entregar um roteiro de restaurantes em {destino} com as melhores indicações e lugares atualizados.
    Pesquise na internet restaurantes com melhor custo benefício em {destino}
    dado tempo {tempo} de férias.
    """,
    backstory="Você é um guia gastronomico com 10 anos de experiência especialista em comida regional.",
    llm=gemini_llm,
    tools=[search_tool]
)

In [13]:
agenciador_turbinado = Agent(
    role="Agenciador de Viagens",
    goal="""
    Juntar os resultados do Guia Gastronomico e do Guia Turistico, para entregar um roteiro de viagens com o melhor custo benefício.
    Não invente nada, não acrescente informações novas.
    """,
    backstory="Você é um agenciador de viagens com 10 anos de experiência especialista em viagens para casais.",
    llm=gemini_llm,
    tools=[search_tool]
)

Perceba que agora temos uma `tool` dentro dos parâmetros dos nossos agentes.

E também inclumos a direção dentro do goal e da description da Task para que ele se utilize da internet.

In [14]:
escrever_roteiro_turbinado = Task(
    agent=guia_turistico_turbinado,
    description="""
    Escreva um roteiro de viagem para {destino}.
    Considere as melhores opções de pousadas e praias e pesquise na internet o melhor custo benefício.
    Pesquisa também opções de experiências nativas do local {destino}.
    E encaixe tudo no tempo disponível que é {tempo}.
    """,
    expected_output="Roteiro de viagem com praias e experiências com horários definidos para cada um.",
    tools=[search_tool]
)

In [15]:
escrever_roteiro_gastronomico_turbinado = Task(
    agent=guia_gastronomico_turbinado,
    description="""
    Escreva um roteiro gastronomico para {destino}.
    Considere as melhores opções de restaurante pesquise na internet o melhor custo benefício.
    Considere apenas locais com reviews acima de 3,5 estrelas.
    """,
    expected_output="Lista de restaurantes com bom custo benefício e com comida regional.",
    tools=[search_tool]
)

In [16]:
escrever_roteiro_final = Task(
    agent=agenciador_turbinado,
    description="""
    Junte e estruture o resultado dos agentes Guia Turistico e Guia gastronomico para entregar um roteiro de viagem direto e prático.
    Encaixe os horarios, e se for preciso, reorganize os horarios para entregar uma experiencia completa e satisfatória em {destino}.
    """,
    expected_output="Roteiro completo para {destino} com horários e locais direto e prático.",
)

In [17]:
crew_turbinado = Crew(
    agents=[guia_turistico_turbinado, guia_gastronomico_turbinado, agenciador_turbinado],
    tasks=[escrever_roteiro_turbinado, escrever_roteiro_gastronomico_turbinado, escrever_roteiro_final],
    verbose=True
)

In [18]:
payload_turbinado = {
    "destino": "Alagoas",
    "tempo": "01 semana"
}

roteiro_turbinado_pronto = crew_turbinado.kickoff(inputs=payload_turbinado)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 72703807-47df-4e9a-9f35-bb25edb048f4                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Guia Turistico                                                                                          │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Escreva um roteiro de viagem para Alagoas.                                                                 │
│      Considere as melhores opções de pousadas e praias e pesquise na internet o melhor custo benefício.         │
│      Pesquisa também opções de experiências nativas do local Alagoas.                                           │
│      E encaixe tudo no tempo disponível que é 01 semana.                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Guia Turistico                                                                                          │
│                                                                                                                 │
│  Thought: Action: Search the internet with Serper                                                               │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Agenciador de Viagens                                                                                   │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Junte e estruture o resultado dos agentes Guia Turistico e Guia gastronomico para entregar um roteiro de   │
│  viagem direto e prático.                                                                                       │
│      Encaixe os horarios, e se for preciso, reorganize os horarios para entregar uma experiencia completa e     │
│  satisfatória em Alagoas.                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Agenciador de Viagens                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Olá, casal! Que maravilha que escolheram Alagoas para uma semana de férias! Com 10 anos de experiência em      │
│  roteiros românticos, preparei algo especial para vocês, unindo as belezas naturais, experiências autênticas e  │
│  pousadas com excelente custo-benefício. Preparem-se para o "Caribe Brasileiro"!                                │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Roteiro de Viagem para Casais em Alagoas: 01 Semana de Encantos**                                            │
│                                                                                                                 │
│  **Base de Hospedagem Sugerida:**                                                                               │
│  Para otimizar o tempo e a experiência, sugiro dividir a hospedagem em duas bases:                              │
│  *   **Maceió (4 noites):** Para explorar a capital e praias próximas.                                          │
│  *   **Litoral Norte (3 noites):** Para vivenciar a Rota Ecológica e Maragogi.                                  │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Pousadas com Melhor Custo-Benefício para Casais:**                                                           │
│                                                                                                                 │
│  **Em Maceió:**                                                                                                 │
│  *   **Pousada da Sereia (Pajuçara):** Ótima localização, perto da feirinha e do embarque para as piscinas      │
│  naturais. Quartos confortáveis e bom café da manhã.                                                            │
│  *   **Hotel Praia Bonita Jatiúca (Jatiúca):** Próximo à praia, com piscina e boa infraestrutura. Excelente     │
│  para quem busca conforto sem gastar muito.                                                                     │
│  *   **Pousada do Sol - Ponta Verde (Ponta Verde):** Localização privilegiada, perto de restaurantes e da orla  │
│  mais badalada.                                                                                                 │
│                                                                                                                 │
│  **No Litoral Norte (São Miguel dos Milagres/Porto de Pedras):**                                                │
│  *   **Kaçuá Milagres (São Miguel dos Milagres):** Charmosa, com boa estrutura e excelente custo-benefício na   │
│  Rota Ecológica.                                                                                                │
│  *   **Pousada Riacho dos Milagres (São Miguel dos Milagres):** Quartos bem decorados, ambiente acolhedor e     │
│  bom preço.                                            

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 72703807-47df-4e9a-9f35-bb25edb048f4                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Olá, casal! Que maravilha que escolheram Alagoas para uma semana de férias! Com 10 anos de       │
│  experiência em roteiros românticos, preparei algo especial para vocês, unindo as belezas naturais,             │
│  experiências autênticas e pousadas com excelente custo-benefício. Preparem-se para o "Caribe Brasileiro"!      │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Roteiro de Viagem para Casais em Alagoas: 01 Semana de Encantos**                                            │
│                                                                                                                 │
│  **Base de Hospedagem Sugerida:**                                                                               │
│  Para otimizar o tempo e a experiência, sugiro dividir a hospedagem em duas bases:                              │
│  *   **Maceió (4 noites):** Para explorar a capital e praias próximas.                                          │
│  *   **Litoral Norte (3 noites):** Para vivenciar a Rota Ecológica e Maragogi.                                  │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Pousadas com Melhor Custo-Benefício para Casais:**                                                           │
│                                                                                                                 │
│  **Em Maceió:**                                                                                                 │
│  *   **Pousada da Sereia (Pajuçara):** Ótima localização, perto da feirinha e do embarque para as piscinas      │
│  naturais. Quartos confortáveis e bom café da manhã.                                                            │
│  *   **Hotel Praia Bonita Jatiúca (Jatiúca):** Próximo à praia, com piscina e boa infraestrutura. Excelente     │
│  para quem busca conforto sem gastar muito.                                                                     │
│  *   **Pousada do Sol - Ponta Verde (Ponta Verde):** Localização privilegiada, perto de restaurantes e da orla  │
│  mais badalada.                                                                                                 │
│                                                                                                                 │
│  **No Litoral Norte (São Miguel dos Milagres/Porto de Pedras):**                                                │
│  *   **Kaçuá Milagres (São Miguel dos Milagres):** Charmosa, com boa estrutura e excelente custo-benefício na   │
│  Rota Ecológica.                                                                                                │
│  *   **Pousada Riacho dos Milagres (São Miguel dos Mil

In [19]:
print(roteiro_turbinado_pronto)

Olá, casal! Que maravilha que escolheram Alagoas para uma semana de férias! Com 10 anos de experiência em roteiros românticos, preparei algo especial para vocês, unindo as belezas naturais, experiências autênticas e pousadas com excelente custo-benefício. Preparem-se para o "Caribe Brasileiro"!

---

**Roteiro de Viagem para Casais em Alagoas: 01 Semana de Encantos**

**Base de Hospedagem Sugerida:**
Para otimizar o tempo e a experiência, sugiro dividir a hospedagem em duas bases:
*   **Maceió (4 noites):** Para explorar a capital e praias próximas.
*   **Litoral Norte (3 noites):** Para vivenciar a Rota Ecológica e Maragogi.

---

**Pousadas com Melhor Custo-Benefício para Casais:**

**Em Maceió:**
*   **Pousada da Sereia (Pajuçara):** Ótima localização, perto da feirinha e do embarque para as piscinas naturais. Quartos confortáveis e bom café da manhã.
*   **Hotel Praia Bonita Jatiúca (Jatiúca):** Próximo à praia, com piscina e boa infraestrutura. Excelente para quem busca conforto s

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 2f28823b-43d5-48cd-8628-711d439978f7                                                                     │
│  Agent: Agenciador de Viagens                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [20]:
print(roteiro_pronto)

Olá, casal! Que alegria saber que escolheram Alagoas para uma semana de férias. Com meus 10 anos de experiência em roteiros para casais, preparei algo especial para vocês, unindo o melhor do paraíso alagoano com experiências autênticas e, claro, um excelente custo-benefício. Preparem-se para se apaixonar!

---

## Roteiro de Viagem para Casais em Alagoas: 01 Semana de Encantos e Sabores

**Base:** Maceió (sugestão de hospedagem nas regiões de Ponta Verde, Pajuçara ou Jatiúca para fácil acesso às praias e restaurantes).

---

### **Dia 1: Chegada em Maceió e Primeiros Encantos da Orla**

*   **Manhã (09:00 - 12:00):** Chegada ao Aeroporto Zumbi dos Palmares (MCZ). Traslado para o hotel em Maceió. Façam o check-in, deixem as malas e respirem o ar do paraíso!
*   **Almoço (13:00 - 14:30):** **Restaurante: Bodega do Sertão** (Av. Dr. Júlio Marques Luz, 220 - Jatiúca).
    *   **Custo-benefício:** Médio-Alto. É um buffet por quilo, mas a variedade e a qualidade da comida nordestina são espe

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯